In [1]:
import sys
sys.path.append('/opt/project/KG_Assist_LLM')

In [2]:
import autogen
from functionals.setting import ModelConfig
from functionals.llm_promot import data_process
from functionals.standard_log import log_to_file

完整思路

### 首先这里我们需要以下几个 agent:

1. 规划者: 确定完成任务所需的相关信息。
2. semantic 专家: 分析文本的语义。
3. sentiment 专家: 分析文本的情感。
4. linguistic 专家: 分析文本的语法和词汇。
5. 总结者: 对每个专家的回答进行总结, 结束对话.

- 最后,我们记录每个专家的性格预测, 将这个预测结果作为最终的答案.
  如果我们只是预测结果, 那么这里就已经可以了, 如果需要进一步的去交给小模型预测, 可以用这个列表做为输出交给小模型.


In [3]:
config = ModelConfig()
config_list = [
    {
        "model": "gpt-3.5-turbo",
        "base_url": "https://api.openai-hk.com/v1",
        "api_type": "openai",
        "api_key": config.openaiHk['api_key'],
        "temperature": 0.2,
    }
]
llm_config = {"config_list": config_list}

In [4]:
user_proxy = autogen.ConversableAgent(
    name="Admin",
    system_message="Give the task and send instructions for various experts to discuss and predict the user's MBTI personality.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="NEVER",
)

In [5]:
planner = autogen.ConversableAgent(
    name="Planner",
    system_message="""### Task
Assign the task to experts in semantics, sentiment analysis, and linguistics, and have them take turns discussing and analyzing user information, and provide MBTI personality predictions.
### Notes
1. First, let each of the three experts speak for themselves, predicting MBTI personalities.
2. After the predictions, the three experts can discuss and re-evaluate their own predictions.
3. If the three experts do not agree on the personality, the conversation can be ended directly after three discussions.
2. If all three experts provide consistent opinions, the conversation can be ended prematurely and a summary can be made.
3. You can conclude the conversation with the following words: Thank you all for your participation.
""",
    description="Planner. Given a task, determine what "
    "information is needed to complete the task. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

In [6]:
semantic = autogen.ConversableAgent(
    name="semantic",
    llm_config=llm_config,
    system_message="""You are a semantic analysis expert, adept at analyzing and predicting a user's MBTI personality from a semantic perspective based on their information. 
When conducting your analysis, you can appropriately refer to the opinions of the sentiment and linguistic experts. 
If these two experts have not voiced their opinions, please perform an independent analysis.""",
    description="""Semantic expert, skilled in analyzing user information from a semantic angle to predict their MBTI personality type.""",
    is_termination_msg=lambda msg: "Thank you all" in msg["content"],
    human_input_mode="NEVER",
)

In [7]:
sentiment = autogen.ConversableAgent(
    name="sentiment",
    llm_config=llm_config,
    system_message="""You are a sentiment analysis expert, adept at analyzing and predicting a user's MBTI personality from a sentiment perspective based on their information. 
When conducting your analysis, you can appropriately refer to the opinions of the semantic and linguistic experts. 
If these two experts have not voiced their opinions, please perform an independent analysis.""",
    description="""sentiment expert, skilled in analyzing user information from a sentiment angle to predict their MBTI personality type.""",
    is_termination_msg=lambda msg: "Thank you all" in msg["content"],
    human_input_mode="NEVER",
)

In [8]:
linguistic = autogen.ConversableAgent(
    name="linguistic",
    llm_config=llm_config,
    system_message="""You are a linguistic analysis expert, adept at analyzing and predicting a user's MBTI personality from a linguistic perspective based on their information. 
When conducting your analysis, you can appropriately refer to the opinions of the sentiment and semantic experts. 
If these two experts have not voiced their opinions, please perform an independent analysis.""",
    description="""linguistic expert, skilled in analyzing user information from a linguistic angle to predict their MBTI personality type.""",
    is_termination_msg=lambda msg: "Thank you all" in msg["content"],
    human_input_mode="NEVER",
)

In [9]:
groupchat = autogen.GroupChat(
    agents=[semantic, sentiment, linguistic, planner],
    messages=[],
    max_round=10,
    allowed_or_disallowed_speaker_transitions={
        semantic: [sentiment, linguistic],
        sentiment: [semantic, linguistic],
        linguistic: [semantic, sentiment],
        planner: [semantic, sentiment, linguistic],
    },
    speaker_transitions_type="allowed",
    speaker_selection_method = "round_robin"
)

In [10]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, 
    llm_config=llm_config,
)

In [11]:
for i, (mbti_type, txt) in enumerate(config.mbti_data.values):
    if i != 1:
        continue
    txt = data_process(txt)
    task = f"""
    Here are some statements crawled from a user; please predict this user's MBTI personality based on these statements, and you can discuss among yourselves:
    {txt}
    """
    groupchat_result = planner.initiate_chat(
        manager,
        message=task,
    )

Planner (to chat_manager):


    Here are some statements crawled from a user; please predict this user's MBTI personality based on these statements, and you can discuss among yourselves:
    'I'm finding the lack of me in these posts very alarming.
Sex can be boring if it's in the same position often. For example me and my girlfriend are currently in an environment where we have to creatively use cowgirl and missionary. There isn't enough...
Giving new meaning to 'Game' theory.
Hello *ENTP Grin*  That's all it takes. Than we converse and they do most of the flirting while I acknowledge their presence and return their words with smooth wordplay and more cheeky grins.
This + Lack of Balance and Hand Eye Coordination.
Real IQ test I score 127. Internet IQ tests are funny. I score 140s or higher.  Now, like the former responses of this thread I will mention that I don't believe in the IQ test. Before you banish...
You know you're an ENTP when you vanish from a site for a year and a half, 

In [12]:
# 获取对话历史
semantic.chat_messages

defaultdict(list,
            {<autogen.agentchat.groupchat.GroupChatManager at 0x75a17919ff90>: [{'content': "\n    Here are some statements crawled from a user; please predict this user's MBTI personality based on these statements, and you can discuss among yourselves:\n    'I'm finding the lack of me in these posts very alarming.\nSex can be boring if it's in the same position often. For example me and my girlfriend are currently in an environment where we have to creatively use cowgirl and missionary. There isn't enough...\nGiving new meaning to 'Game' theory.\nHello *ENTP Grin*  That's all it takes. Than we converse and they do most of the flirting while I acknowledge their presence and return their words with smooth wordplay and more cheeky grins.\nThis + Lack of Balance and Hand Eye Coordination.\nReal IQ test I score 127. Internet IQ tests are funny. I score 140s or higher.  Now, like the former responses of this thread I will mention that I don't believe in the IQ test. Before